# This is Week 3 Coursera Capstone Project for the IBM Data Science Professional Certificate.

## First Task 
### I used BeautifulSoup and urllib packages for the scraping and making a request to the wiki page, lxml parser for parsing the page into BeautifulSoup Object that contains the tags and table tag to get the content.

In [5]:
!pip install bs4

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=68f2d65cc4f439a4403c31b1f13eef9e493dece6b9b15e0dbd944138f95c10b7
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [7]:
# import relevant libraries 
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [15]:
# import url
url = urlopen(r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [16]:
# create BeautifulSoup object
soup = BeautifulSoup(url, "lxml")

In [17]:
table = soup.table

#### It's time to loop through rows in the table and get the data from each row appended into lists.

In [18]:
#create lists and append relevant data from table
PostalCode=[]
Borough=[]
Neighborhood=[]

for tr in table.find_all('tr')[1:]:
    PostalCode.append(tr.find_all('td')[0].get_text())
    Borough.append(tr.find_all('td')[1].get_text())
    Neighborhood.append(tr.find_all('td')[2].get_text().strip('\n'))

#### Create a Dataframe.

In [19]:
df = pd.DataFrame({
        'PostalCode':PostalCode,
        'Borough':Borough,
        'Neighborhood':Neighborhood
    })

df.head()

PostalCode  \
0              \nM1BScarborough(Malvern / Rouge)\n\n   
1  \nM1CScarborough(Rouge Hill / Port Union / Hig...   
2  \nM1EScarborough(Guildwood / Morningside / Wes...   
3                       \nM1GScarborough(Woburn)\n\n   
4                    \nM1HScarborough(Cedarbrae)\n\n   

                                  Borough  \
0                   \nM2BNot assigned\n\n   
1                   \nM2CNot assigned\n\n   
2                   \nM2ENot assigned\n\n   
3                   \nM2GNot assigned\n\n   
4  \nM2HNorth York(Hillcrest Village)\n\n   

                                        Neighborhood  
0                      M3BNorth York(Don Mills)North  
1     M3CNorth York(Don Mills)South(Flemingdon Park)  
2                                    M3ENot assigned  
3                                    M3GNot assigned  
4  M3HNorth York(Bathurst Manor / Wilson Heights ...

#### Appending the data, while also assigning possible "Not assigned" Neighborhoods with the borough data.

In [20]:
#transform data
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
#import transformed data into DataFrame
df=pd.DataFrame(table_contents)

In [31]:
df

PostalCode                                            Borough  \
0          M3A                                         North York   
1          M4A                                         North York   
2          M5A                                   Downtown Toronto   
3          M6A                                         North York   
4          M7A                                       Queen's Park   
..         ...                                                ...   
98         M8X                                          Etobicoke   
99         M4Y                                   Downtown Toronto   
100        M7Y  East TorontoBusiness reply mail Processing Cen...   
101        M8Y                                          Etobicoke   
102        M8Z                                          Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Regent Park, Harbourfront  
3                     Lawrence Manor, Lawrence Heights  
4                        Ontario Provincial Government  
..                                                 ...  
98       The Kingsway, Montgomery Road, Old Mill North  
99                                Church and Wellesley  
100                                     Enclave of M4L  
101  Old Mill South, King's Mill Park, Sunnylea, Hu...  
102  Mimico NW, The Queensway West, South of Bloor,...  

[103 rows x 3 columns]

In [22]:
df.shape

(103, 3)

## Second Task

In [36]:
import pandas as pd
import io
import requests
url="https://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

dfc = df.join(c.set_index('Postal Code'), on='PostalCode')
dfc.head(11)

PostalCode           Borough                      Neighborhood   Latitude  \
0         M3A        North York                         Parkwoods  43.753259   
1         M4A        North York                  Victoria Village  43.725882   
2         M5A  Downtown Toronto         Regent Park, Harbourfront  43.654260   
3         M6A        North York  Lawrence Manor, Lawrence Heights  43.718518   
4         M7A      Queen's Park     Ontario Provincial Government  43.662301   
5         M9A         Etobicoke                  Islington Avenue  43.667856   
6         M1B       Scarborough                    Malvern, Rouge  43.806686   
7         M3B        North York                   Don Mills North  43.745906   
8         M4B         East York   Parkview Hill, Woodbine Gardens  43.706397   
9         M5B  Downtown Toronto          Garden District, Ryerson  43.657162   
10        M6B        North York                         Glencairn  43.709577   

    Longitude  
0  -79.329656  
1  -79.315572  
2  -79.360636  
3  -79.464763  
4  -79.389494  
5  -79.532242  
6  -79.194353  
7  -79.352188  
8  -79.309937  
9  -79.378937  
10 -79.445073